In [ ]:
#TDA API Greek Options Attribution Tool

### Data Aquisition

- Aquiring the data from TDA's API. 
- I'll have to use my TDA credentials, so I'm hiding them inside my env file and only using them as a specific variable. That way the actual credentials shouldn't be compromised.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import math
import datetime

import tda
import os
import requests
import json
import base64

In [2]:
import env

Polygon.io API key loaded up.
Cboe API key loaded up.
TD Ameritrade API key loaded up.
Coinbase API key loaded up.
All credentials loaded successfully


In [ ]:
# TDA API keys

tda_key = env.tda_api_key
tda_url = env.my_redirect_url

In [3]:
# Cboe LiveVol API keys:

cboe_key = env.cboe_client_id
cboe_secret = env.cboe_client_secret

In [ ]:
requests.get(f'https://api.tdameritrade.com//v1/marketdata/chains?apikey={tda_key}&symbol=AAPL&contractType=CALL&strikeCount=2&includeQuotes=TRUE&strategy=SINGLE&toDate=2021-03-16&expMonth=MAR&optionType=S HTTP/1.1')

In [ ]:
aapl_api_test = requests.get(f'https://api.tdameritrade.com//v1/marketdata/chains?apikey={tda_key}&symbol=AAPL&contractType=CALL&strikeCount=2&includeQuotes=TRUE&strategy=SINGLE&toDate=2021-03-16&expMonth=MAR&optionType=S HTTP/1.1')

In [ ]:
aapl_content = aapl_api_test.content

# aapl_content

In [ ]:
request = requests.get(f'https://api.tdameritrade.com//v1/marketdata/chains?apikey={tda_key}&symbol=AAPL&contractType=CALL&strikeCount=2&includeQuotes=TRUE&strategy=SINGLE&toDate=2021-03-16&expMonth=MAR&optionType=S HTTP/1.1').text
aapl = json.loads(request)
# print(aapl['callExpDateMap'])

In [ ]:
aapl

In [ ]:
aapl['callExpDateMap']

In [ ]:
aapl['callExpDateMap']['2021-03-05:9']['124.0'][0]

In [ ]:
aapl['callExpDateMap']['2021-03-05:13']['129.0']

### Preparing the data

- The goal is to grab just the option greek and volatility data from the option chain. We don't need any other data at this point.
- I'm going to use a Pandas dataframe to help organize my work.

In [ ]:
df = pd.DataFrame(aapl['callExpDateMap']['2021-03-05:9']['124.0'][0], index = [0])

df

In [ ]:
df.columns

In [ ]:
greeks = ['delta', 'gamma', 'theta', 'vega', 'rho']

volatility_info = ['openInterest', 'timeValue', 'theoreticalOptionValue', 'theoreticalVolatility']

basic_info = ['mark', 'totalVolume', 'volatility', 'inTheMoney']

In [ ]:
greeks + basic_info + volatility_info

In [ ]:
columns_list = basic_info + greeks + volatility_info

In [ ]:
df[[c for c in df.columns if c in columns_list]]

In [ ]:
df = df[[c for c in df.columns if c in columns_list]]
df

In [ ]:
# 3.53 to 2.50



### Creating the Auth Code:

In [33]:
# Creating the client ID variables to use in getting an access token:
client_id = env.cboe_client_id
client_secret = env.cboe_client_secret

identity_url = "https://id.livevol.com/connect/token"

encoded = base64.b64encode((client_id + ':' + client_secret).encode())
headers = {"Authorization": "Basic " + encoded.decode('ascii')}
payload = {"grant_type": "client_credentials"}

# Requesting access token
token_data = requests.post(identity_url, data=payload, headers=headers)

if token_data.status_code == 200:
    token = token_data.json()['access_token']
    if len(token) > 0:
        print("Authenticated successfully")
        # Requesting data from API
#         result = requests.get(f'https://api.livevol.com/v1/live/allaccess/market/option-and-underlying-quotes?root={root}&option_type={option_type}&date={date}&min_expiry={min_expiry}&max_expiry={max_expiry}&min_strike={min_strike}&max_strike={max_strike}&symbol={symbol}', headers={"Authorization": "Bearer " + token}, verify=False)
#         print(result.json())
else:
    print("Authentication failed")

Authenticated successfully


#### DataFrame Function

In [31]:
# Example data points used in function:

# root = 'TDOC'
# option_type = 'C'
# date = '2021-01-04'
# min_expiry = '2022-01-21'
# max_expiry = '2022-01-21'
# min_strike = '145'
# max_strike = '145'
# symbol = root


def cboe_data(root, option_type, date, min_expiry, max_expiry, min_strike, max_strike, symbol, header_code):
    
    url = f'https://api.livevol.com/v1/live/allaccess/market/option-and-underlying-quotes?root={root}&option_type={option_type}&date={date}&min_expiry={min_expiry}&max_expiry={max_expiry}&min_strike={min_strike}&max_strike={max_strike}&symbol={symbol}'
    
    cboe_headers = {'Authorization': f'Bearer {header_code}'}
    
    
    # Getting the data from the API:
    stock_get = requests.get(url = url, headers = cboe_headers)
    
    stock_data = json.loads(stock_get.text)
    
    # Creating the Dataframe and cleaning up columns:
    stock_df = pd.DataFrame(stock_data['options'][0], index = [0])
    columns_to_keep = ['root', 'expiry', 'strike', 'option_type', 'timestamp', 'option_mid', 'option_bid', 'option_ask', 'delta', 'gamme', 'vega', 'theta', 'rho']
    
    # Using list comprehension to filter for the columns I want to keep
    stock_df = stock_df[[c for c in stock_df.columns if c in columns_to_keep]]
    
    # Adding the as of date:
    stock_df['as_of_date'] = date
    
    # Returning the df:
    
    return stock_df

In [6]:
# Function using delayed data from Cboe:

def cboe_data_delayed(root, option_type, date, min_expiry, max_expiry, min_strike, max_strike, symbol, header_code):
    
    '''
    Must use the Cboe LiveVol's API formatting for this call. This function essentially fills in the blanks for a live API call using the function arguments.
    '''
    
    url = f'https://api.livevol.com/v1/delayed/allaccess/market/option-and-underlying-quotes?root={root}&option_type={option_type}&date={date}&min_expiry={min_expiry}&max_expiry={max_expiry}&min_strike={min_strike}&max_strike={max_strike}&symbol={symbol}'
    
    cboe_headers = {'Authorization': f'Bearer {header_code}'}
    
    
    # Getting the data from the API:
    stock_get = requests.get(url = url, headers = cboe_headers)
    
    stock_data = json.loads(stock_get.text)
    
    # Creating the Dataframe and cleaning up columns:
    stock_df = pd.DataFrame(stock_data['options'][0], index = [0])
    columns_to_keep = ['root', 'expiry', 'strike', 'option_type', 'timestamp', 'option_mid', 'option_bid', 'option_ask', 'delta', 'gamme', 'vega', 'theta', 'rho']
    
    # Using list comprehension to filter for the columns I want to keep
    stock_df = stock_df[[c for c in stock_df.columns if c in columns_to_keep]]
    
    # Adding the as of date:
    stock_df['as_of_date'] = date
    
    # Returning the df:
    
    return stock_df

### Data Aquisition

In [7]:
# Creating the url and authorization headers:

end_url = 'https://api.livevol.com/v1/delayed/allaccess/market/option-and-underlying-quotes?root=AAPL&option_type=C&date=2021-02-26&min_expiry=2021-03-19&max_expiry=2021-03-19&min_strike=130&max_strike=130&symbol=AAPL'
start_url = 'https://api.livevol.com/v1/delayed/allaccess/market/option-and-underlying-quotes?root=AAPL&option_type=C&date=2021-02-01&min_expiry=2021-03-19&max_expiry=2021-03-19&min_strike=130&max_strike=130&symbol=AAPL'
cboe_headers = {'Authorization': 'Bearer ' + token}


# Loading the start date and end date option data from the API:

r_start = requests.get(url = start_url, headers = cboe_headers)
start_data = json.loads(r_start.content)

r_end = requests.get(url = end_url, headers = cboe_headers)
end_data = json.loads(r_end.content)


In [8]:
start_data

{'options': [{'root': 'AAPL',
   'expiry': '2021-03-19',
   'strike': 130.0,
   'option_type': 'C',
   'timestamp': None,
   'option': 'AAPL210319C00130000',
   'option_mid': 9.95,
   'option_trade_count': None,
   'option_bid': 9.9,
   'option_bid_size': 231,
   'option_ask': 10.0,
   'option_ask_size': 152,
   'option_open': 10.2,
   'option_high': 10.55,
   'option_low': 8.45,
   'option_close': 9.6,
   'option_prev_day_close': 9.41,
   'option_last_trade_price': 9.6,
   'iv': 0.4157,
   'open_interest': 87715,
   'option_volume': 5501,
   'delta': 0.6104,
   'gamma': 0.0194,
   'vega': 0.1824,
   'theta': -0.0816,
   'rho': 0.0}],
 'timestamp': None,
 'symbol': 'AAPL',
 'implied_underlying_bid': None,
 'implied_underlying_ask': None,
 'implied_underlying_bid_size': None,
 'implied_underlying_ask_size': None,
 'implied_underlying_mid': None,
 'underlying_mid': None,
 'underlying_last_trade_price': None,
 'underlying_last_trade_size': None,
 'underlying_bid': None,
 'underlying_ask':

In [9]:
end_data

{'options': [{'root': 'AAPL',
   'expiry': '2021-03-19',
   'strike': 130.0,
   'option_type': 'C',
   'timestamp': None,
   'option': 'AAPL210319C00130000',
   'option_mid': 1.845,
   'option_trade_count': None,
   'option_bid': 1.84,
   'option_bid_size': 108,
   'option_ask': 1.85,
   'option_ask_size': 68,
   'option_open': 1.73,
   'option_high': 2.11,
   'option_low': 1.2,
   'option_close': 1.23,
   'option_prev_day_close': 1.54,
   'option_last_trade_price': 1.23,
   'iv': 0.4212,
   'open_interest': 96747,
   'option_volume': 20705,
   'delta': 0.2616,
   'gamma': 0.0266,
   'vega': 0.0946,
   'theta': -0.095,
   'rho': 0.0}],
 'timestamp': None,
 'symbol': 'AAPL',
 'implied_underlying_bid': None,
 'implied_underlying_ask': None,
 'implied_underlying_bid_size': None,
 'implied_underlying_ask_size': None,
 'implied_underlying_mid': None,
 'underlying_mid': None,
 'underlying_last_trade_price': None,
 'underlying_last_trade_size': None,
 'underlying_bid': None,
 'underlying_ask'

In [10]:
# Creating the save_json function:

def save_json(var_obj, filename):
    '''
    This function saves a python dictionary as a json object in a text file. Purpose is to save my option data calls to save on my API call limits.
    The filename MUST be a string and have the .txt extension, eg: `start_data.csv`. 
    var_obj must be an already existing variable that you want to save to a text file.
    '''
    
#     filename = str(input("Please enter a filename: "))
    
#     var_obj = 
    
    with open(filename, 'w') as outfile:
        json.dump(var_obj, outfile)

In [11]:
save_json(start_data, 'start_data.txt')

In [12]:
save_json(end_data, 'end_data.txt')

## The G/OAT Calculator

In [13]:
# Finding delta, both current and from a past date:

start_delta = start_data['options'][0]['delta']
start_gamma = start_data['options'][0]['gamma']
start_vega = start_data['options'][0]['vega']
start_theta = start_data['options'][0]['theta']
start_rho = start_data['options'][0]['rho']

In [14]:
start_delta, start_gamma, start_vega, start_theta, start_rho

(0.6104, 0.0194, 0.1824, -0.0816, 0.0)

In [15]:
end_delta = end_data['options'][0]['delta']
end_gamma = end_data['options'][0]['gamma']
end_vega = end_data['options'][0]['vega']
end_theta = end_data['options'][0]['theta']
end_rho = end_data['options'][0]['rho']

In [16]:
end_delta, end_gamma, end_vega, end_theta, end_rho

(0.2616, 0.0266, 0.0946, -0.095, 0.0)

In [17]:
# Now the start and end price for this option:

start_price = start_data['options'][0]['option_mid']
end_price = end_data['options'][0]['option_mid']

start_price, end_price, round(start_price - end_price, 2)

(9.95, 1.845, 8.1)

In [ ]:
# Dates:

start_date = 

In [18]:
# Starting and Ending price of the underlying asset, in this case Apple's stock price. 

# We'll use the most recent previous day's stock close price to keep things simple for now:

start_under_price = start_data['underlying_prev_day_close']
end_under_price = end_data['underlying_prev_day_close']



In [19]:
start_data['underlying_prev_day_close']

131.96

In [20]:
start_data

{'options': [{'root': 'AAPL',
   'expiry': '2021-03-19',
   'strike': 130.0,
   'option_type': 'C',
   'timestamp': None,
   'option': 'AAPL210319C00130000',
   'option_mid': 9.95,
   'option_trade_count': None,
   'option_bid': 9.9,
   'option_bid_size': 231,
   'option_ask': 10.0,
   'option_ask_size': 152,
   'option_open': 10.2,
   'option_high': 10.55,
   'option_low': 8.45,
   'option_close': 9.6,
   'option_prev_day_close': 9.41,
   'option_last_trade_price': 9.6,
   'iv': 0.4157,
   'open_interest': 87715,
   'option_volume': 5501,
   'delta': 0.6104,
   'gamma': 0.0194,
   'vega': 0.1824,
   'theta': -0.0816,
   'rho': 0.0}],
 'timestamp': None,
 'symbol': 'AAPL',
 'implied_underlying_bid': None,
 'implied_underlying_ask': None,
 'implied_underlying_bid_size': None,
 'implied_underlying_ask_size': None,
 'implied_underlying_mid': None,
 'underlying_mid': None,
 'underlying_last_trade_price': None,
 'underlying_last_trade_size': None,
 'underlying_bid': None,
 'underlying_ask':

In [37]:
type(start_data['options'][0]['timestamp'])

NoneType

In [21]:
# The calculator estimates the attribution of price changes in an option to each of the main greeks; Delta, Vega, and Theta. This is a test of that idea.

In [22]:
round(start_price - end_price, 2)

8.1

In [23]:
calc_df = pd.DataFrame({"start_date": [start_price, start_delta, start_gamma, start_vega, start_theta, start_rho], "end_date": [end_price, end_delta, end_gamma, end_vega, end_theta, end_rho]})
calc_df

,start_date,end_date
0,9.9500,1.8450
1,0.6104,0.2616
2,0.0194,0.0266
3,0.1824,0.0946
4,-0.0816,-0.0950
5,0.0000,0.0000


In [25]:
calc_df.start_date

0    9.9500
1    0.6104
2    0.0194
3    0.1824
4   -0.0816
5    0.0000
Name: start_date, dtype: float64

In [26]:
# I need to transpose the df so that the greeks are columns and the dates are rows.

In [29]:
pd.DataFrame({"price": start_price, "delta": start_delta}, index = [0])

,price,delta
0,9.95,0.6104


### Black-Scholes Options Model

   
     
      

**Call Formula:**

$$ C = S_{t}N(d_{1}) - Ke^{-rt}N(d_{2}) $$

**where:**

$$d_1 = \frac{\ln \left(\frac{S}{K} \right) + \left(r + \frac{\sigma^2}{2} \right)(T - t)}{\sigma \sqrt{T - t}}$$


In [32]:
# Now, the key is to figure out how to utilize this within the calculator.